We create Food graph with intreuction form https://neo4j.com/graphgists/dd3dedcf-c377-4575-84f4-4d0d30b2a4c5/

In [1]:
pip install neo4j


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 12.8 MB/s 
  Created wheel for neo4j: filename=neo4j-5.2.1-py3-none-any.whl size=249499 sha256=458cde597c0d355022642404d6eea7684dd5b3ac6ef2019013ee4e73e324e503
  Stored in directory: /root/.cache/pip/wheels/b3/8c/94/ad2ec76fee32d266ed8954ba221bc716e468f24d1f07d98f92
Successfully built neo4j


In [2]:
#Import biblioteki do tworzenia i obsługi modelu word2vec
from gensim.models import Word2Vec
#Import bibliteki niezbędnej do komunikacji z bazą danych w Neo4j
from neo4j import GraphDatabase, unit_of_work

In [3]:
host = 'bolt://54.144.195.217:7687'
user = 'neo4j'
password = 'bridge-stares-stopper'
driver = GraphDatabase.driver(host,auth=(user, password))

In [4]:
unit_of_work(timeout=240000)

<function neo4j.work.query.unit_of_work.<locals>.wrapper(f)>

CALL gds.graph.project(
  'food',            
  ['Ingredient','Recipe'],             
  'CONTAINS_INGREDIENT'               
)

In [27]:
# Define random walk query
random_walks_query = """
CALL gds.randomWalk.stream( 'food', {
  walkLength: 10,
  walksPerNode: 15,
  inOutFactor:0.6,
  returnFactor:1.0,
  concurrency: 4,
  randomSeed: 18
})
YIELD nodeIds
RETURN [id in nodeIds | gds.util.asNode(id).name] as walks
"""
# Fetch data from neo4j
with driver.session() as session:
    walks = session.run(random_walks_query)
    # Train the word2vec model
    clean_walks = [row['walks'] for row in walks]
model = Word2Vec(clean_walks, sg=1, window=5, size=100)


model.save("foodEmbedding_default.model")

In [28]:
model.most_similar('chicken')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('olive oil', 0.9998606443405151),
 ('garlic clove', 0.9998373985290527),
 ('plain flour', 0.9998284578323364),
 ('butter', 0.9998241066932678),
 ('lime', 0.9998089075088501),
 ('cherry tomato', 0.9998054504394531),
 ('potato', 0.9997981786727905),
 ('butternut squash', 0.9997943639755249),
 ('cucumber', 0.9997928738594055),
 ('chicken stock', 0.9997913837432861)]

In [31]:
model.most_similar('sugar')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('onion', 0.999841570854187),
 ('plain flour', 0.9998328685760498),
 ('egg', 0.9998183250427246),
 ('garlic clove', 0.9998116493225098),
 ('carrot', 0.9998116493225098),
 ('butter', 0.9998053312301636),
 ('milk', 0.9998040199279785),
 ('nutmeg', 0.9998010993003845),
 ('potato', 0.9997948408126831),
 ('cinnamon', 0.9997947216033936)]

In [17]:
model.save("foodEmbedding.model")